In [1]:
%matplotlib inline

import os
import sys
genre_class_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
print("Loading Genre_Classification system from path", genre_class_path)
sys.path.append(genre_class_path)

import numpy as np
import pandas as pd
import tensorflow as tf
import json
import random

Loading Genre_Classification system from path /mnt/d/UNIV NOTES/Y4S1/DSA4199/genre_classification


2022-11-09 22:50:23.804754: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-09 22:50:23.805333: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [6]:
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk import download
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer

import string
import re
import rpy2.robjects as ro

download('averaged_perceptron_tagger')

ModuleNotFoundError: No module named 'rpy2'

In [3]:
dataset_wkspc = os.path.join(genre_class_path, 'workspace', 'gb_input')
data_path = os.path.join(dataset_wkspc, 'data_index.json')

with open(data_path) as f:
    data_json = json.load(f)

In [5]:
index = []
for entry in data_json:
    index.append(entry['index'])

In [6]:
index[:10]

['10002',
 '10005',
 '10008',
 '10046',
 '10067',
 '10094',
 '10109',
 '10124',
 '1014',
 '10234']

In [7]:
data_json[0]

{'index': '10002',
 'indexfile': 'Gutenberg/cache/epub/10002/pg10002.rdf',
 'title': 'the house on the borderland',
 'author': 'hodgson, william hope',
 'subjects': ['Science fiction'],
 'bodyfile': 'Gutenberg/aleph.gutenberg.org/1/0/0/0/10002/10002.zip',
 'body': 'project gutenberg\'s the house on the borderland, by william hope hodgson\r\n\r\nthis ebook is for the use of anyone anywhere at no cost and with\r\nalmost no restrictions whatsoever.  you may copy it, give it away or\r\nre-use it under the terms of the project gutenberg license included\r\nwith this ebook or online at www.gutenberg.org\r\n\r\n\r\ntitle: the house on the borderland\r\n\r\nauthor: william hope hodgson\r\n\r\nrelease date: november 10, 2003 [ebook #10002]\r\nlast updated: january 19, 2009\r\n\r\nlanguage: english\r\n\r\ncharacter set encoding: ascii\r\n\r\n*** start of this project gutenberg ebook the house on the borderland ***\r\n\r\n\r\n\r\n\r\nproduced by suzanne shell, sjaani and pg distributed proofreade

In [8]:
dummy = data_json[0]['body']

In [9]:
#dummy = re.sub(r'[^\w\s]', '', dummy)
dummy = dummy.lower()
dummy = word_tokenize(dummy)
dummy2 = []

ps = PorterStemmer()
for word in dummy:
    dummy2.append(ps.stem(word))
#dummy = re.sub(r'\r\n', ' ', dummy)
#dummy = " ".join(dummy.split())
#dummy = dummy.rstrip()

In [10]:
print(dummy)

['project', 'gutenberg', "'s", 'the', 'house', 'on', 'the', 'borderland', ',', 'by', 'william', 'hope', 'hodgson', 'this', 'ebook', 'is', 'for', 'the', 'use', 'of', 'anyone', 'anywhere', 'at', 'no', 'cost', 'and', 'with', 'almost', 'no', 'restrictions', 'whatsoever', '.', 'you', 'may', 'copy', 'it', ',', 'give', 'it', 'away', 'or', 're-use', 'it', 'under', 'the', 'terms', 'of', 'the', 'project', 'gutenberg', 'license', 'included', 'with', 'this', 'ebook', 'or', 'online', 'at', 'www.gutenberg.org', 'title', ':', 'the', 'house', 'on', 'the', 'borderland', 'author', ':', 'william', 'hope', 'hodgson', 'release', 'date', ':', 'november', '10', ',', '2003', '[', 'ebook', '#', '10002', ']', 'last', 'updated', ':', 'january', '19', ',', '2009', 'language', ':', 'english', 'character', 'set', 'encoding', ':', 'ascii', '*', '*', '*', 'start', 'of', 'this', 'project', 'gutenberg', 'ebook', 'the', 'house', 'on', 'the', 'borderland', '*', '*', '*', 'produced', 'by', 'suzanne', 'shell', ',', 'sjaani

In [11]:
print(dummy2)

['project', 'gutenberg', "'s", 'the', 'hous', 'on', 'the', 'borderland', ',', 'by', 'william', 'hope', 'hodgson', 'thi', 'ebook', 'is', 'for', 'the', 'use', 'of', 'anyon', 'anywher', 'at', 'no', 'cost', 'and', 'with', 'almost', 'no', 'restrict', 'whatsoev', '.', 'you', 'may', 'copi', 'it', ',', 'give', 'it', 'away', 'or', 're-us', 'it', 'under', 'the', 'term', 'of', 'the', 'project', 'gutenberg', 'licens', 'includ', 'with', 'thi', 'ebook', 'or', 'onlin', 'at', 'www.gutenberg.org', 'titl', ':', 'the', 'hous', 'on', 'the', 'borderland', 'author', ':', 'william', 'hope', 'hodgson', 'releas', 'date', ':', 'novemb', '10', ',', '2003', '[', 'ebook', '#', '10002', ']', 'last', 'updat', ':', 'januari', '19', ',', '2009', 'languag', ':', 'english', 'charact', 'set', 'encod', ':', 'ascii', '*', '*', '*', 'start', 'of', 'thi', 'project', 'gutenberg', 'ebook', 'the', 'hous', 'on', 'the', 'borderland', '*', '*', '*', 'produc', 'by', 'suzann', 'shell', ',', 'sjaani', 'and', 'pg', 'distribut', 'proof

In [12]:
len(dummy2)

65744

In [13]:
[x for x in dummy2 if x not in string.punctuation]

['project',
 'gutenberg',
 "'s",
 'the',
 'hous',
 'on',
 'the',
 'borderland',
 'by',
 'william',
 'hope',
 'hodgson',
 'thi',
 'ebook',
 'is',
 'for',
 'the',
 'use',
 'of',
 'anyon',
 'anywher',
 'at',
 'no',
 'cost',
 'and',
 'with',
 'almost',
 'no',
 'restrict',
 'whatsoev',
 'you',
 'may',
 'copi',
 'it',
 'give',
 'it',
 'away',
 'or',
 're-us',
 'it',
 'under',
 'the',
 'term',
 'of',
 'the',
 'project',
 'gutenberg',
 'licens',
 'includ',
 'with',
 'thi',
 'ebook',
 'or',
 'onlin',
 'at',
 'www.gutenberg.org',
 'titl',
 'the',
 'hous',
 'on',
 'the',
 'borderland',
 'author',
 'william',
 'hope',
 'hodgson',
 'releas',
 'date',
 'novemb',
 '10',
 '2003',
 'ebook',
 '10002',
 'last',
 'updat',
 'januari',
 '19',
 '2009',
 'languag',
 'english',
 'charact',
 'set',
 'encod',
 'ascii',
 'start',
 'of',
 'thi',
 'project',
 'gutenberg',
 'ebook',
 'the',
 'hous',
 'on',
 'the',
 'borderland',
 'produc',
 'by',
 'suzann',
 'shell',
 'sjaani',
 'and',
 'pg',
 'distribut',
 'proofre

In [16]:
from collections import Counter
Counter([x for x in dummy2 if x not in string.punctuation])

Counter({'project': 92,
         'gutenberg': 30,
         "'s": 58,
         'the': 3852,
         'hous': 128,
         'on': 238,
         'borderland': 7,
         'by': 156,
         'william': 5,
         'hope': 15,
         'hodgson': 5,
         'thi': 283,
         'ebook': 27,
         'is': 203,
         'for': 345,
         'use': 33,
         'of': 1837,
         'anyon': 7,
         'anywher': 2,
         'at': 352,
         'no': 141,
         'cost': 5,
         'and': 1750,
         'with': 434,
         'almost': 50,
         'restrict': 2,
         'whatsoev': 3,
         'you': 87,
         'may': 45,
         'copi': 25,
         'it': 1090,
         'give': 20,
         'away': 77,
         'or': 159,
         're-us': 2,
         'under': 40,
         'term': 27,
         'licens': 16,
         'includ': 16,
         'onlin': 4,
         'www.gutenberg.org': 3,
         'titl': 3,
         'author': 3,
         'releas': 6,
         'date': 7,
         'novemb':

In [17]:
len([x for x in dummy2 if x not in string.punctuation])

55196

In [18]:
stemmed_df = []
subjects = {'Adventure stories': 1, 'Detective and mystery stories': 2, 'Historical fiction': 3, 'Love stories': 4,
           'Science fiction': 5, 'Western stories': 6}

In [19]:
ps = PorterStemmer()
for entry in data_json:
    body = entry['body'].lower()
    body = word_tokenize(body)
    word_list = []
    for word in body:
        word_list.append(ps.stem(word))
    for subject in entry['subjects']:
        if subject in subjects:
            subject_code = subjects[subject]
            break
    word_list = [x for x in word_list if x not in string.punctuation]
    stemmed_df.append({'index': entry['index'], 'title': entry['title'], 'word_list': Counter(word_list), 
                      'body_length': entry['body_length'], 'subjects': subject_code})

In [21]:
stemmed_df[0]

{'index': '10002',
 'title': 'the house on the borderland',
 'word_list': Counter({'project': 92,
          'gutenberg': 30,
          "'s": 58,
          'the': 3852,
          'hous': 128,
          'on': 238,
          'borderland': 7,
          'by': 156,
          'william': 5,
          'hope': 15,
          'hodgson': 5,
          'thi': 283,
          'ebook': 27,
          'is': 203,
          'for': 345,
          'use': 33,
          'of': 1837,
          'anyon': 7,
          'anywher': 2,
          'at': 352,
          'no': 141,
          'cost': 5,
          'and': 1750,
          'with': 434,
          'almost': 50,
          'restrict': 2,
          'whatsoev': 3,
          'you': 87,
          'may': 45,
          'copi': 25,
          'it': 1090,
          'give': 20,
          'away': 77,
          'or': 159,
          're-us': 2,
          'under': 40,
          'term': 27,
          'licens': 16,
          'includ': 16,
          'onlin': 4,
          'www.gutenbe

In [22]:
stemmed_word_list = stemmed_df.copy()

In [24]:
for entry in stemmed_word_list:
    new_dict = {}
    for key in entry['word_list'].keys():
        new_dict[str(key)] = entry['word_list'][key]
    entry['word_list'] = new_dict

In [25]:
len(stemmed_word_list)

3889

In [26]:
stemmed_word_list[0]

{'index': '10002',
 'title': 'the house on the borderland',
 'word_list': {'project': 92,
  'gutenberg': 30,
  "'s": 58,
  'the': 3852,
  'hous': 128,
  'on': 238,
  'borderland': 7,
  'by': 156,
  'william': 5,
  'hope': 15,
  'hodgson': 5,
  'thi': 283,
  'ebook': 27,
  'is': 203,
  'for': 345,
  'use': 33,
  'of': 1837,
  'anyon': 7,
  'anywher': 2,
  'at': 352,
  'no': 141,
  'cost': 5,
  'and': 1750,
  'with': 434,
  'almost': 50,
  'restrict': 2,
  'whatsoev': 3,
  'you': 87,
  'may': 45,
  'copi': 25,
  'it': 1090,
  'give': 20,
  'away': 77,
  'or': 159,
  're-us': 2,
  'under': 40,
  'term': 27,
  'licens': 16,
  'includ': 16,
  'onlin': 4,
  'www.gutenberg.org': 3,
  'titl': 3,
  'author': 3,
  'releas': 6,
  'date': 7,
  'novemb': 4,
  '10': 3,
  '2003': 3,
  '10002': 1,
  'last': 70,
  'updat': 2,
  'januari': 2,
  '19': 1,
  '2009': 1,
  'languag': 2,
  'english': 2,
  'charact': 1,
  'set': 22,
  'encod': 1,
  'ascii': 4,
  'start': 23,
  'produc': 5,
  'suzann': 2,
  'sh

In [27]:
with open('stemmed_word_list.json', 'w') as f:
    json.dump(stemmed_word_list, f)

In [4]:
dataset_wkspc = os.path.join(genre_class_path, 'workspace', 'gb_input')
stem_path = os.path.join(dataset_wkspc, 'stemmed_word_list.json')

with open(stem_path) as f:
    stemmed_word_list = json.load(f)

In [5]:
stem_combi = []
for entry in stemmed_word_list:
    for key in entry['word_list'].keys():
        stem_combi.append(key)

In [6]:
stem_combi = list(set(stem_combi))

In [7]:
len(stem_combi)

486945

In [8]:
df_cols = stem_combi + ['subjects']

In [9]:
df = pd.DataFrame(columns=df_cols)

In [10]:
row = 0
for entry in stemmed_word_list:
    for word in stem_combi:
        if word in entry['word_list'].keys():
            df.loc[row, word] = entry['word_list'][word]
        else :
            df.loc[row, word] = 0
    df.loc[row, 'subjects'] = entry['subjects']
    row = row + 1

KeyboardInterrupt: 

In [11]:
df.head()

,brownrigg,reed-thatch,geeve-a,pacqu,samarcand_,'aughtiest,broad-skirt,intramur,o'er-burthen,nine-kilolumen,...,frosty-blu,ratl,tessera,ginerelli,sleep-wrack,livery-and-sal,285139-f.,_bennington's_,may-hap,subjects
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2


In [12]:
df.shape

(676, 486946)

In [13]:
df_dummy = df.copy()

In [ ]:
index = []
for entry in stemmed_word_list:
    index.append(entry['index'])

In [ ]:
df.index = index
df.head()

In [ ]:
df.to_csv('stemmed_word_full.csv')

In [5]:
preproc = [x[0] for x in ro.r('''
library(tm)
library(jsonlite)
data <- data_json
corpus <- VCorpus(VectorSource(data$body))
corpus <- tm_map(corpus, removeNumbers)
corpus <- tm_map(corpus, removePunctuation)
corpus <- tm_map(corpus, function(x) gsub("\r\n", " ", x))
corpus <- tm_map(corpus, stripWhitespace)
corpus <- tm_map(corpus, tolower)
corpus <- tm_map(corpus, PlainTextDocument)''')]

NameError: name 'ro' is not defined

In [ ]:
def mytokenizer(x):
    return [y for y in x.split() if len(y) > 2]

# Full document-term matrix
cv = CountVectorizer(tokenizer=mytokenizer)
X = cv.fit_transform(preproc)
X
# <1181x3289 sparse matrix of type '<type 'numpy.int64'>'
#   with 8980 stored elements in Compressed Sparse Column format>

# Sparse terms removed
cv2 = CountVectorizer(tokenizer=mytokenizer, min_df=0.005)
X2 = cv2.fit_transform(preproc)
X2
# <1181x309 sparse matrix of type '<type 'numpy.int64'>'
#   with 4669 stored elements in Compressed Sparse Column format>